In [1]:
import sys, json

sys.path.insert(0, "../")

from lib import *

Welcome to JupyROOT 6.28/08


In [2]:
user_input = {
    "config_file": "hd_1x2x6/hd_1x2x6_config",
    "root_file": ["Marley"],
    "rewrite": True,
    "debug": True,
}
info = json.load(open("../config/" + user_input["config_file"] + ".json", "r"))
data_filter = {
    "max_energy": 20,
    "min_energy": 0,
    "pre_nhits": 3,
    "primary": True,
    "neutron": True,
}
true, data, filter_idx = compute_root_workflow(
    user_input, info, data_filter, workflow="CALIBRATION", debug=user_input["debug"]
)
rprint(
    "-> Found %i electron candidates out of %i events!"
    % (len(filter_idx), data["Event"].size)
)

Loading Marley data...
Truth:
        -> WARNING: Key Event not found in true tree!
        -> WARNING: Key Flag not found in true tree!
Reco:
        -> Found Event   <class 'int'>
        -> Found Flag    <class 'int'>
        -> Found Generator       <class 'int'>
        -> Found NHits   <class 'int'>
        -> Found Ind0NHits       <class 'int'>
        -> Found Ind1NHits       <class 'int'>
        -> Found Ind0Charge      <class 'int'>
        -> Found Ind1Charge      <class 'int'>
        -> Found Ind0dT          <class 'int'>
        -> Found Ind1dT          <class 'int'>
        -> WARNING: Key AdjClNum not found in reco tree!
        -> WARNING: Key AdjOpFlashNum not found in reco tree!
        -> Found TNuE    <class 'float'>
        -> WARNING: Key ElectronE not found in reco tree!
        -> WARNING: Key GammaE not found in reco tree!
        -> WARNING: Key NeutronP not found in reco tree!
        -> WARNING: Key VisEnergy not found in reco tree!
        -> Found Time    <class 'float'>
        -> Found Charge          <class 'float'>
        -> Found Primary         <class 'bool'>

Output()

Output()

-> Finished computing data!

-> Found 24058 electron candidates out of 219147 events!

In [9]:
# Plot the calibration workflow
acc = 50
factor = 1
fit = {"color":"grey","threshold":0.4,"trimm":2, "spec_type":"max", "print":True, "opacity": 1}
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Electron Lifetime", "Energy Resolution", "Charge Calibration", "Energy Resolution")
)

fit["func"] = "exponential"
fig, corr_popt, perr = get_hist2d_fit(
    np.abs(data["Time"][filter_idx]),
    (data["Charge"] / (factor * data["ElectronE"]))[filter_idx],
    fig,
    idx=(1,1),
    per=[1,99],
    acc=acc,
    fit=fit,
    density=True,
    zoom=True,
    debug=False,
)

# Energy computation
data["Correction"] = np.exp(np.abs(data["Time"]) / corr_popt[1])
data["Energy"] = data["Charge"] * data["Correction"] / corr_popt[0]

x,y,h = get_hist2d(np.abs(data["Energy"][filter_idx]),100*(data["ElectronE"][filter_idx] - data["Energy"][filter_idx])/data["Energy"][filter_idx],density=True,acc=acc)
fig.add_trace(
    go.Heatmap(
        x=x,
        y=y,
        z=h.T,
        coloraxis="coloraxis",
    ),row=1,col=2
)

fit["func"] = "linear"
fig, reco_popt, perr = get_hist2d_fit(
    factor * data["ElectronE"][filter_idx],
    data["Energy"][filter_idx],
    fig,
    idx=(2,1),
    per=[1,99],
    acc=acc,
    fit=fit,
    zoom=True,
    debug=False,
)

fit["func"] = "gauss"
fit["color"] = "red"
fig, res_popt, perr = get_hist1d_fit(
    (data["Energy"] / (factor * data["ElectronE"]))[filter_idx],
    fig,
    idx=(2,2),
    per=[1,99],
    acc=acc,
    fit=fit,
    debug=False,
)

fig.update_layout(
    coloraxis=dict(colorscale="Turbo", colorbar=dict(title="Norm")),
    showlegend=False,
    xaxis1_title="Time [tick]",
    xaxis2_title="True Electron Energy [MeV]",
    xaxis3_title="True Electron Energy [MeV]",
    xaxis4_title="Corr. Charge/Energy",
    yaxis1_title="Charge/Energy (e⁻) [ADC x tick/MeV]",
    yaxis2_title="(Truth - Reco)/Truth Energy [%]",
    yaxis3_title="Reco Electron Energy [MeV]",
    yaxis4_title="Norm.",
)

fig = format_coustom_plotly(
    fig, fontsize=18, matches=(None, None), figsize=(2000,1000), title="Cluster-based electron charge calibration"
)
fig.show()

INFO: 
Fit parameter Amplitude: 133.169372 +/- 0.606423
Fit parameter Tau: 19148.344640 +/- 696.435737

INFO: 
Fit parameter Slope: 1.066201 +/- 0.004309
Fit parameter Intercept: -0.548932 +/- 0.044419

INFO: 
Fit parameter Amplitude: 0.086671 +/- 0.004277
Fit parameter Mean: 0.984700 +/- 0.003655
Fit parameter Sigma: 0.064151 +/- 0.003655

In [7]:
config = user_input["config_file"].split("/")[0]
if not os.path.exists("../config/" + config + "/" + config + "_calib/"):
    os.makedirs("../config/" + config + "/" + config + "_calib/")

fig = format_coustom_plotly(fig, fontsize=18, matches=(None, None))
if not os.path.exists("../images/calibration/%s_calibration/" % config):
    os.makedirs("../images/calibration/%s_calibration/" % config)

if user_input["rewrite"]:
    fig.write_image(
        "../images/calibration/%s_calibration/%s_calibration.png" % (config, config),
        width=2400,
        height=1080,
    )
    rprint(
        "-> Saved images to ../images/calibration/%s_calibration/%s_calibration.png"
        % (config, config)
    )

# Save the true energy fit parameters to a json file
if (
    not os.path.exists(f"../config/{config}/{config}_calib/{config}_charge_correction.json"
    )
    or user_input["rewrite"]
):
    with open(f"../config/{config}/{config}_calib/{config}_charge_correction.json","w") as f:
        json.dump({"CHARGE_AMP": corr_popt[0], "ELECTRON_TAU": corr_popt[1]}, f)
    
    rprint(f"-> Saved calibration parameters to ../config/{config}/{config}_calib/{config}_charge_correction.json")

else:
    rprint(f"-> Found ../config/{config}/{config}_calib/{config}_charge_correction.json")
    rprint(f"-> Please set rewrite to True to overwrite the file")

-> Saved images to ../images/calibration/hd_1x2x6_calibration/hd_1x2x6_calibration.png

-> Saved calibration parameters to ../config/hd_1x2x6/hd_1x2x6_calib/hd_1x2x6_charge_correction.json